In [35]:
import re
import spacy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

from deep_translator import GoogleTranslator

In [36]:
data = pd.read_csv (r'C:\Users\willy\Desktop\TCC_classificador\Dados\tcepi_decisoes_ate_10_2020.csv', sep=";")   
df = pd.DataFrame(data, columns= ['ementa','assunto_agrupado'])
dfLicitacao = df.loc[df['assunto_agrupado'] == "LICITACAO"] 
dfPessoal = df.loc[df['assunto_agrupado'] == "PESSOAL"]
dfOutros = df.loc[df['assunto_agrupado'] == "OUTROS"]
dfDespesa = df.loc[df['assunto_agrupado'] == "DESPESA"]
dfProcessual = df.loc[df['assunto_agrupado'] == "PROCESSUAL"]
dfPrestacaoDeContas = df.loc[df['assunto_agrupado'] == "PRESTACAO DE CONTAS"]
dfContrato = df.loc[df['assunto_agrupado'] == "CONTRATO"]
dfPrevidencia = df.loc[df['assunto_agrupado'] == "PREVIDENCIA"]
dfAgentePolitico = df.loc[df['assunto_agrupado'] == "AGENTE POLITICO"]
dfResponsabilidade = df.loc[df['assunto_agrupado'] == "RESPONSABILIDADE"]

df['assunto_agrupado'].value_counts()

LICITACAO              122
PESSOAL                118
OUTROS                 106
DESPESA                 65
PROCESSUAL              62
PRESTACAO DE CONTAS     46
CONTRATO                43
PREVIDENCIA             35
AGENTE POLITICO         31
RESPONSABILIDADE        24
Name: assunto_agrupado, dtype: int64

In [37]:
def removerSinaisDePontuacao(textos):
    textosLimpos = []
    nlp = spacy.load('pt_core_news_lg')
    for texto in textos:
        if(len(texto) <= 5000):
            texto = re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ: ]', ' ', texto)
            doc = nlp(texto)
            text_aux = []
            for token in doc:
                if token.is_space == False:
                    text_aux.append(token.text.lower()) 
            textosLimpos.append(" ".join(text_aux))
    return textosLimpos

def traduzirParaIngles(textos):
    textosEmIngles = []
    for texto in textos:
        traducao = GoogleTranslator(source='portuguese', target='english').translate(texto)
        textosEmIngles.append(traducao)
    return textosEmIngles

def traduzirParaPortugues(textos):
    textosEmPortugues = []
    for texto in textos:
        traducao = GoogleTranslator(source='english', target='portuguese').translate(texto)
        textosEmPortugues.append(traducao)
    return textosEmPortugues

def gerarSinonimos(texto):
    aug = naw.SynonymAug(aug_src='wordnet')
    textoAlterado = aug.augment(texto)
    return textoAlterado

In [38]:
despesas = removerSinaisDePontuacao(dfDespesa['ementa'])
despesasAssuntos = dfDespesa['assunto_agrupado'][:len(despesas)].tolist()

processual = removerSinaisDePontuacao(dfProcessual['ementa'])
processualAssuntos = dfProcessual['assunto_agrupado'][:len(processual)].tolist()

prestacaoDeContas = removerSinaisDePontuacao(dfPrestacaoDeContas['ementa'])
prestacaoDeContasAssuntos = dfPrestacaoDeContas['assunto_agrupado'][:len(prestacaoDeContas)].tolist()

contratos = removerSinaisDePontuacao(dfContrato['ementa'])
contratoAssuntos = dfContrato['assunto_agrupado'][:len(contratos)].tolist()

previdencia = removerSinaisDePontuacao(dfPrevidencia['ementa'])
previdenciaAssuntos = dfPrevidencia['assunto_agrupado'][:len(previdencia)].tolist()

responsabilidades = removerSinaisDePontuacao(dfResponsabilidade['ementa'])
responsabilidadesAssuntos = dfResponsabilidade['assunto_agrupado'][:len(responsabilidades)].tolist()

agentePolitico = removerSinaisDePontuacao(dfAgentePolitico['ementa'])
agentePoliticoAssuntos = dfAgentePolitico['assunto_agrupado'][:len(agentePolitico)].tolist()

In [39]:
ementas = despesas + processual + prestacaoDeContas + contratos + previdencia + responsabilidades + agentePolitico
assuntos = despesasAssuntos + processualAssuntos + prestacaoDeContasAssuntos + contratoAssuntos + previdenciaAssuntos + responsabilidadesAssuntos + agentePoliticoAssuntos

In [40]:
print(ementas[0])

prefeitura municipal irregularidades em pagamento de diárias insuficiência no histórico de empenhos procedência parcial da denúncia advertência apensamento 1 a ausência de indícios de pagamento de diárias como complementação salarial ou do desvio de finalidade na sua concessão não possui gravidade suficiente para macular o procedimento licitatório 2 a falta de informações precisas no histórico dos empenhos enseja a procedência da denúncia tendo em vista a relevância desses dados para a efetiva transparência e controle dos gastos públicos


In [41]:
print(assuntos[0])

DESPESA


In [42]:
print(len(assuntos))
print(len(ementas))

305
305


In [45]:
despesasEmIngles = traduzirParaIngles(despesas)
print("D")
processualEmIngles = traduzirParaIngles(processual)
print("P")
prestacaoDeContasEmIngles = traduzirParaIngles(prestacaoDeContas)
print("PC")
contratosEmIngles = traduzirParaIngles(contratos)
print("C")
previdenciaEmIngles = traduzirParaIngles(previdencia)
print("PR")
agentePoliticoEmIngles = traduzirParaIngles(agentePolitico)
print("A")
responsabilidadesEmIngles = traduzirParaIngles(responsabilidades)
print("R")
print(responsabilidadesEmIngles[0])

D
P
PC
C
PR
A
R
reconsideration appeal sesapi municipality of são pedro do piauí agreement No. 559 2009 technical assistance project in the organization of the service that integrates the municipal health system in view of the detailed analysis of the records, there is no way to exclude the responsibility and/or penalty of the managers responsible for the conduct that is described in the records there is clearly a deviation from the purpose in which the responsibility must lie with the head of the municipal executive


In [46]:
ementasEmIngles = despesasEmIngles + processualEmIngles + prestacaoDeContasEmIngles + contratosEmIngles + previdenciaEmIngles + responsabilidadesEmIngles + agentePoliticoEmIngles

In [48]:
print(ementasEmIngles[0])

city ​​hall irregularities in the payment of daily rates insufficiency in the history of commitments partial origin of the complaint warning attachment 1 the absence of evidence of payment of daily rates such as salary supplementation or deviation of purpose in its concession is not sufficiently serious to tarnish the bidding procedure 2 the lack accurate information in the history of the efforts gives rise to the accusation in view of the relevance of these data for the effective transparency and control of public spending


In [49]:
despesasSinonimos = []
processualSinonimos = []
prestacaoDeContasSinonimos = []
contratosSinonimos = []
previdenciaSinonimos = []
agentePoliticoSinonimos = []
responsabilidadesSinonimos = []

for texto in despesasEmIngles:
    despesasSinonimos.append(gerarSinonimos(texto))
    
for texto in processualEmIngles:
    processualSinonimos.append(gerarSinonimos(texto))
    
for texto in prestacaoDeContasEmIngles:
    prestacaoDeContasSinonimos.append(gerarSinonimos(texto))
    
for texto in contratosEmIngles:
    contratosSinonimos.append(gerarSinonimos(texto))
    
for texto in previdenciaEmIngles:
    previdenciaSinonimos.append(gerarSinonimos(texto))
    
for texto in agentePoliticoEmIngles:
    agentePoliticoSinonimos.append(gerarSinonimos(texto))
    
for texto in responsabilidadesEmIngles:
    responsabilidadesSinonimos.append(gerarSinonimos(texto))
    
novasDespesas = traduzirParaPortugues(despesasSinonimos)
novasProcessual = traduzirParaPortugues(processualSinonimos)
novasPrestacaoDeContas = traduzirParaPortugues(prestacaoDeContasSinonimos)
novosContratos = traduzirParaPortugues(contratosSinonimos)
novasPrevidencias = traduzirParaPortugues(previdenciaSinonimos)
novosAgentesPoliticos = traduzirParaPortugues(agentePoliticoSinonimos)
novasResponsabilidades = traduzirParaPortugues(responsabilidadesSinonimos)

print(novasResponsabilidades)

['recurso de reconsideração sesapi município de são pedro do piauí concord no 559 2009 assistência técnica trabalhista na organização do serviço que integra o sistema municipal de saúde tendo em vista a análise elaborada dos prontuários, não há direcionamento para excluir a responsabilidade e / ou pena dos gestores responsáveis \u200b\u200bpela conduta que conste dos autos houver, compreensivelmente, desvio da finalidade em que a responsabilidade deve recair sobre o chefe do diretor executivo municipal', 'contratos de dever assinados direção anterior agora analisada preparação o gestor não é imune para ocorrências não motivadas pelo mesmo e decorrem de contrato previamente assinado', 'controle interno da prefeitura sublocação de veículos sem mandato legal uso de veículos inadequados ao transporte escolar Guerrilha pagamento de despesas com locação de veículos para unidade de fita escolar e atendimento multifacetado fracionamento de despesas cadastramento extemporâneo de dispensa de lic

In [50]:
novasEmentasEmIngles = despesasSinonimos + processualSinonimos + prestacaoDeContasSinonimos + contratosSinonimos + previdenciaSinonimos + responsabilidadesSinonimos + agentePoliticoSinonimos
novasEmentas = novasDespesas + novasProcessual + novasPrestacaoDeContas + novosContratos + novasPrevidencias + novasResponsabilidades + novosAgentesPoliticos

In [51]:
print(len(ementas))
print(len(ementasEmIngles))
print(len(novasEmentasEmIngles))
print(len(novasEmentas))
print(len(assuntos))

305
305
305
305
305


In [52]:
dataFrame = pd.DataFrame(list(zip(ementas, ementasEmIngles, novasEmentasEmIngles, novasEmentas, assuntos)),
               columns =['ementa', 'ementa_em_ingles', 'nova_ementa_em_ingles', 'nova_ementa','assunto_agrupado'])

In [53]:
dataFrame.to_excel("./dados/novas_decisoes.xlsx")  
dataFrame.to_csv('./dados/novas_decisoes.csv', sep=';', encoding='utf-8', index=False)